In [6]:

import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [7]:

def get_coordinate(addr):
    browser = webdriver.Chrome(executable_path='chromedriver',options=options)
    browser.get("http://www.map.com.tw/")
    search = browser.find_element_by_id("searchWord")
    search.clear()
    search.send_keys(addr)
    browser.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
    time.sleep(2)
    iframe = browser.find_elements_by_tag_name("iframe")[1]
    browser.switch_to.frame(iframe)
    coor_btn = browser.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
    coor_btn.click()
    coor = browser.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
    coor = coor.text.strip().split(" ")
    lat = coor[-1].split("：")[-1]
    log = coor[0].split("：")[-1]
    browser.quit()
    return (lat, log)

In [13]:

get_coordinate("臺北市中正區延平南路96號")

('25.043928', '121.509781')

In [ ]:
import csv
# 開啟輸出的 CSV 檔案
with open('output2.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    # 寫入一列資料
    writer.writerow(['緯緯度'])

    # 開啟 CSV檔案
    with open('監視器.csv', newline='',encoding="utf-8") as csvfile:

                # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile)
        for row in rows:
            print(get_coordinate(row[5]))
            writer.writerow(get_coordinate(row[5]))

('24.98972977', '121.57004896')
('24.98972977', '121.57004896')


In [36]:
import requests

if __name__ == "__main__":
    res = requests.get(
        "http://api.map.com.tw/net/GraphicsXY.aspx?",
        data={
            'search_class': 'address',
            'searchkey': 'abc',
            'fun': 'funB',
            'SearchWord': '新北市中和區中正路716號'
        },
        headers={
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
            'Connection':'keep-alive',
            'Host':'api.map.com.tw',
            'Referer':'http://api.map.com.tw/API_Sample/Sample/GetAddressCoordinates.html',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
            }
    )
    print(res.text)

funB({"locate":{"lat":"24.997123","lng":"121.485925","no":"暫無資料","bheigh2":"18","bcount2":"166","baddr2":"新北市","bname2":"中和區","village":"平河里","road":"中正路716號","msg":"Success","precision":"100"}})


In [3]:
import requests
import re
import csv
import json
import time


def addrTo(addr):
    res = requests.get(
        "http://api.map.com.tw/net/GraphicsXY.aspx?",
        data={
            'search_class': 'address',
            'searchkey': 'abc',
            'fun': 'funB',
            'SearchWord': addr
        },
        headers={
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
            'Connection': 'close',
            'Host': 'api.map.com.tw',
            'Referer': 'http://api.map.com.tw/API_Sample/Sample/GetAddressCoordinates.html',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
        }
    )
    return res.text


if __name__ == "__main__":

    with open('output.csv', 'w', encoding='UTF-8', newline='') as write_file:
        writer = csv.writer(write_file)
        writer.writerow(['地址', 'Lat緯度', 'Lng經度'])
        count = 1

        with open('臺北市政府警察局錄影監視系統設置區位.csv', 'r', newline='') as csvfile:
            # 讀取 CSV 檔案內容
            rows = csv.reader(csvfile)
            for row in rows:
                # 捕捉到地址
                addr = '臺北市'+row[2]
                # print(row[3])

                # 透過捕捉到地址放入函式中取得json資料
                json_data = addrTo(addr)
                # print(json)
                # time.sleep(0)

                # 利用正則來擷取我們所需的經緯度資料
                # \d+\.?\d* 小數點形式的數字
                try:
                    m_Lat = re.search(r'(?<=lat":").*?(\d+\.?\d*)', json_data)
                    m_Lng = re.search(r'(?<=lng":").*?(\d+\.?\d*)', json_data)
                    print(addr)
                    print(m_Lat.group())
                    print(m_Lng.group())
                    print(count)
                    count += 1
                    writer.writerow([addr, m_Lat.group(), m_Lng.group()])
                except Exception as e:
                    print(json_data)
                    print(e)

臺北市民生西路231號
25.057155
121.515713
1
臺北市民生西路257號
25.057133
121.514517
2
臺北市寧夏路55號燈桿
25.057508
121.515496
3
臺北市興城街31號前燈桿
25.058502
121.517676
4
臺北市錦西街63號
25.059856
121.515708
5
臺北市興城街36號旁
25.058289
121.517604
6
臺北市民生西路171巷27號前燈桿
25.057865
121.516859
7
臺北市興城街75號前燈桿
25.05964
121.517448
8
臺北市寧夏路67號旁
25.058392
121.515217
9
臺北市民權西路136號前燈桿
25.062746
121.516403
10
臺北市承德路2段與民權西路東南角
25.062848
121.518274
11
臺北市寧夏路55號旁
25.057508
121.515496
12
臺北市重慶北路2段171號〈家樂福〉
25.059023
121.513805
13
臺北市重慶北路2段171號〈家樂福〉
25.059023
121.513805
14
臺北市民生西路49號旁(雙連捷運2號出口右側號誌)
25.057786
121.520436
15
臺北市興城街75號前燈桿
25.05964
121.517448
16
臺北市錦西街178號
25.059724
121.515903
17
臺北市錦西街178號
25.059724
121.515903
18
臺北市寧夏路77號右前方
25.058639
121.515135
19
臺北市承德路2段132號(分隔島)
25.05921
121.517983
20
臺北市歸綏街88-1號旁
25.058673
121.516269
21
臺北市歸綏街63號
25.059303
121.517372
22
臺北市歸綏街83巷口
25.059251
121.516357
23
臺北市雙連街53巷28號旁燈桿
25.05928
121.519949
24
臺北市民生西路63號旁
25.057725
121.520117
25
臺北市萬全街42號
25.058591
121.520012
26
臺北市萬全街42號
25.058591
121.520012
2

臺北市民族西路200號前
25.068465
121.51437
209
臺北市北市大同區重慶北路3段269號前路口監控
24.98972977
121.57004896
210
臺北市承德路3段與酒泉街口照路口
25.073948
121.519709
211
臺北市重慶北路3段165號前人行道上
25.068791
121.513919
212
臺北市民族西路163號
25.068732
121.515551
213
臺北市酒泉街54-1號
25.071917
121.516021
214
臺北市承德路3段223號燈桿
25.070108
121.518635
215
臺北市民族西路31巷18號旁燈桿
25.069199
121.519529
216
臺北市庫倫街13巷4弄3號前
25.072778
121.517272
217
臺北市重慶北路3段320號前地下道
25.07474
121.513581
218
臺北市重慶北路3段320號前地下道
25.07474
121.513581
219
臺北市重慶北路3段310號
25.073901
121.513544
220
臺北市延平北路290號前
25.090473
121.501293
221
臺北市敦煌路175巷口
25.076793
121.510814
222
臺北市延平北路4段197號
25.073986
121.511607
223
臺北市重慶北路3段312巷28號前
25.074058
121.512821
224
臺北市環河北路2段377號
25.077427
121.509134
225
臺北市酒泉街與環河北路路口號誌(污水處理廠旁)
24.98972977
121.57004896
226
臺北市延平北路4段157號
25.073125
121.511404
227
臺北市酒泉街150號前
25.072025
121.512119
228
臺北市重慶北路3段282號牆上
25.073102
121.513594
229
臺北市敦煌路131號牆上
25.076649
121.512554
230
臺北市敦煌路167號
25.076676
121.511208
231
臺北市迪化路2段265號
25.074175
121.511514
232
臺北市重慶北路3段336號側牆
25.076013
1

臺北市南京西路462號
25.053581
121.507648
408
臺北市南京西路434之1號
25.053204
121.508945
409
臺北市民樂街32號
25.056441
121.510738
410
臺北市長安西路與西寧北路口(西南角)
25.05201
121.510446
411
臺北市城區醫院鄭州路40號門前號
24.98972977
121.57004896
412
臺北市南京西路412巷1弄8號
25.052995
121.509672
413
臺北市延平北路50巷1號旁燈桿
25.090473
121.501293
414
臺北市南京西路390號前燈桿
25.053552
121.510405
415
臺北市南京西路211號前
25.053966
121.511134
416
臺北市南京西路與延平北路口
25.054163
121.514744
417
臺北市長安西路340-1號燈桿
25.05193
121.510598
418
臺北市延平北路1段106號旁燈桿
25.052886
121.511674
419
臺北市北市大同區延平北路2段61巷21號燈桿路口監控
24.98972977
121.57004896
420
臺北市延平北路2段與民生西路(西北角號誌桿)
25.060637
121.511327
421
臺北市西寧北路86巷7號
25.055289
121.508643
422
臺北市南京西路239巷39號
25.053765
121.509009
423
臺北市環河北路141號前
25.090473
121.501293
424
臺北市延平北路2段60巷21號
25.05548
121.51097
425
臺北市民樂街31號旁
25.054494
121.511054
426
臺北市南京西路233巷16號旁燈桿
25.05425
121.510406
427
臺北市南京西路239巷1號右側
25.053693
121.509923
428
臺北市迪化街1段14巷38號
25.054134
121.509296
429
臺北市迪化街1段44號口燈桿路口監控
25.054951
121.509972
430
臺北市迪化街1段64號
25.055422
121.509901
431
臺北市貴德街30-1號對面燈桿
25.0

臺北市太原路79巷底
25.051682
121.516022
613
臺北市太原路113號旁燈桿
25.052629
121.515909
614
臺北市長安西路78巷3弄13號旁燈桿
25.049713
121.519663
615
臺北市長安西路196號旁號誌桿
25.051103
121.515543
616
臺北市華陰街,太原路口
24.98972977
121.57004896
617
臺北市華陰街119號
25.050049
121.516517
618
臺北市華陰街119號
25.050049
121.516517
619
臺北市華陰街87號牆上
25.049771
121.517711
620
臺北市長安西路78巷3弄2號後方防火巷燈桿
25.049742
121.519187
621
臺北市長安西路52巷口(全家便利超商旁)
25.049637
121.519629
622
臺北市長安西路138巷5弄口
25.05014
121.517623
623
臺北市太原路22巷3號前燈桿
25.049946
121.514683
624
臺北市太原路44號前
25.050442
121.515084
625
臺北市承德、長安西路口西南角
24.98972977
121.57004896
626
臺北市長安西路78巷4弄2號右旁燈桿
25.049778
121.519021
627
臺北市長安西路138巷5弄口
25.05014
121.517623
628
臺北市承德路1段19巷17號對面牆上
25.050362
121.517539
629
臺北市承德路與市民大道(西北角燈桿)
25.120845
121.500174
630
臺北市長安西路170號側牆
25.05096
121.516124
631
臺北市長安西路78巷1弄2之1號
25.049915
121.519683
632
臺北市重慶北路1段15巷口
25.05007
121.514207
633
臺北市承德路1段26號旁燈桿
25.050592
121.516726
634
臺北市華陰街203號壁掛(巷南監視巷內)
25.050482
121.514524
635
臺北市長安西路147號前號誌桿
25.051516
121.514793
636
臺北市鄭州路與承德路東端分隔島號誌桿
25.

臺北市西園路1段177號
25.034681
121.499412
825
臺北市西園路1段314之2號對面的前分隔島上
25.033361
121.497847
826
臺北市西園路1段282巷口
25.033832
121.499064
827
臺北市廣州街與南寧路口(分隔島上)
24.98972977
121.57004896
828
臺北市大理街42巷7號前燈桿
25.034196
121.500381
829
臺北市艋舺大道77號
25.033794
121.503246
830
臺北市北市萬華區西園路1段203巷和大理街42巷路口監控
24.98972977
121.57004896
831
臺北市和平西路3段104之1號
25.035093
121.500616
832
臺北市中華路2段2號前(分隔島上)
25.03417
121.505561
833
臺北市康定路344號
25.035047
121.501052
834
臺北市大理街42-2號
25.034482
121.500054
835
臺北市和平西路3段30巷5之1號
25.034109
121.504058
836
臺北市和平西路3段88巷口分隔島
25.034948
121.502307
837
臺北市和平西路3段68巷口
25.034937
121.503097
838
臺北市大理街2號
25.034475
121.503215
839
臺北市和平西路3段52巷30號
25.03384
121.503236
840
臺北市艋舺大道120巷口(中央分隔島上)
25.032911
121.502343
841
臺北市和平西路2段151號左側對面(艋舺大道中央分隔島前)
25.033809
121.50508
842
臺北市西園路1段226號前(分隔島上)
25.035152
121.498996
843
臺北市南寧路與廣洲街路口
25.03654
121.504814
844
臺北市南寧路48巷12號
25.035174
121.50461
845
臺北市三水街22巷口
25.035204
121.50404
846
臺北市康定路356號
25.034734
121.500992
847
臺北市和平西路2段151號前(中央分隔島)
25.033809
121.50508
848
臺北市和平

...

臺北市康樂街20巷2號前
25.068355
121.617989
14570
臺北市康寧路一段195巷4弄4號
25.078996
121.596482
14571
臺北市成功路四段223巷6弄13號對面
25.083936
121.597198
14572
臺北市成功路四段223巷8弄16號對面
25.083757
121.597317
14573
臺北市星雲街49號
25.078805
121.591734
14574
臺北市環山路二段53巷口(55號旁)
25.084504
121.574324
14575
臺北市內湖路一段602號前
25.078733
121.578185
14576
臺北市港華街48號前
25.083027
121.574707
14577
臺北市港墘路127巷18號旁
25.077746
121.577206
14578
臺北市寧夏路與錦西街東南角號誌
25.060797
121.514348
14579
臺北市大龍街91巷33號旁
25.067381
121.517395
14580
臺北市敦煌路24巷12弄2號旁
25.074473
121.519148
14581
臺北市迪化街二段161號前燈桿
25.067583
121.510463
14582
臺北市南京西路241號前
25.053673
121.510139
14583
臺北市南京西路107巷50號對面
25.055143
121.515542
14584
臺北市平陽街46號
25.055428
121.515252
14585
臺北市承德路二段53巷36號
25.055274
121.519552
14586
臺北市太原街與平陽街東南角燈桿(桿號094)
25.057177
121.516969
14587
臺北市環河南路一段與成都路口(西南角)
25.043042
121.500534
14588
臺北市隆昌街31號
25.04122
121.50511
14589
臺北市西園路一段212巷28-1號
25.035739
121.498296
14590
臺北市梧州街49號前
25.03695
121.497703
14591
臺北市梧州街49號前
25.03695
121.497703
14592
臺北市環河南路一段344號(水濂宮前)
25.040551
121.

臺北市陽金公路下馬槽橋第24號燈桿
25.179482
121.582228
14765
臺北市陽金公路下馬槽橋第114號燈桿
25.179482
121.582228
14766
臺北市陽金公路下馬槽橋第114號燈桿
25.179482
121.582228
14767
臺北市平菁街20號
25.138091
121.571536
14768
臺北市至善路三段199號對面燈桿
25.119819
121.579622
14769
臺北市至善路三段371巷20弄燈桿(N001-13L)
25.123582
121.582229
14770
臺北市至善路三段76號前燈桿(01)
25.113322
121.566645
14771
臺北市至善路三段71巷13號旁燈桿
25.118686
121.567979
14772
臺北市尊賢街264巷25號
25.117665
121.508068
14773
臺北市尊賢街264巷25號
25.117665
121.508068
14774
臺北市自強街120巷19號
25.110183
121.515981
14775
臺北市文林北路80巷59弄17號前
25.106641
121.51904
14776
臺北市行義路158巷口
25.131058
121.529196
14777
臺北市行義路137巷1弄巷口(上坡)
25.129016
121.526614
14778
臺北市行義路112巷1號
25.127794
121.529274
14779
臺北市裕民一路與裕民二路口
25.11679
121.516204
14780
臺北市東華街二段202號旁
25.12042
121.510119
14781
臺北市中央北路一段1號對面圓環
25.133399
121.501253
14782
臺北市東昇路195號轉彎處065燈桿
25.160625
121.532329
14783
臺北市東昇路39-1號路口旁燈桿(027)
25.149603
121.524307
14784
臺北市登山路69~81之43號巷口對面
25.146412
121.507306
14785
臺北市泉源路至妙寺前
25.147854
121.530652
14786
臺北市復興三路521巷16弄臨34號往淡水方向100公尺轉彎處燈桿(285)
25

臺北市松江路558號前
25.067609
121.533001
14949
臺北市建國北路三段與民權東路二段西北角近第一殯儀館
25.067628
121.533303
14950
臺北市民權東路三段復興北路西北角
25.06237
121.540858
14951
臺北市建國北路二段131號對面行人號誌桿
25.0602
121.53732
14952
臺北市復興北路354號前
25.060256
121.543929
14953
臺北市民生東路三段55號前分隔島號誌桿
25.058049
121.540968
14954
臺北市民生東路三段1號對面行人號誌桿
25.058147
121.537309
14955
臺北市民權東路三段3號(民權龍江西北角號誌桿)
25.062625
121.541017
14956
臺北市民族東路538號對面行人號誌桿
25.067032
121.543218
14957
臺北市建國北路三段91號前
25.065734
121.536765
14958
臺北市復興北路490號前
25.065226
121.544109
14959
臺北市民權東路三段與建國北路(東北角)行人號誌桿靠建國北路
25.06237
121.540858
14960
臺北市民權東路二段137號前燈桿
25.062672
121.535046
14961
臺北市建國北路二段119號斜前方紅綠燈上
25.059614
121.537308
14962
臺北市北安路516之3號前分隔島
25.079694
121.545795
14963
臺北市北安路516之3號前分隔島
25.079694
121.545795
14964
臺北市明水路(大直橋下)中央分隔島
25.085633
121.553036
14965
臺北市北安路與明水路口
25.085633
121.553036
14966
臺北市基湖路與樂群一路口分隔島
25.082337
121.565508
14967
臺北市樂群三路與敬業三路東南角
25.082642
121.560648
14968
臺北市忠孝東路三段98號前分隔島號誌
25.04162
121.536448
14969
臺北市復興南路一段66號前
25.04442
121.543619
14970
臺北市復興南路一段66號前
25.0

臺北市忠孝東路五段345巷口東北角
25.04133
121.574107
15133
臺北市忠孝東路五段743巷口(781號旁三角分隔島)
25.043951
121.580975
15134
臺北市信義路六段63號前
25.035327
121.576682
15135
臺北市松德路133號前中央分隔島
25.037292
121.574977
15136
臺北市光復南路76號前分隔島號誌
25.044769
121.557542
15137
臺北市光復南路68號前燈桿
25.045314
121.557597
15138
臺北市忠孝東路五段159號
25.041238
121.568485
15139
臺北市松仁路、忠孝東路五段口西側分隔島號誌桿
25.041045
121.568449
15140
臺北市忠孝東路及基隆路口東側分隔島燈桿
25.053176
121.616287
15141
臺北市忠孝東路及基隆路口東側分隔島燈桿
25.053176
121.616287
15142
臺北市百齡橋西側橋頭右側(015燈桿.閘門旁)
24.98972977
121.57004896
15143
臺北市延平北路七段18-2號前燈桿
25.095187
121.49791
15144
臺北市延平北路六段350-1號前中央分隔島
25.090711
121.501319
15145
臺北市延平北路六段350-1號前中央分隔島
25.090711
121.501319
15146
臺北市重慶北路四段152號
25.083814
121.512289
15147
臺北市延平北路六段400號與倫等街口分隔島號誌桿
25.091513
121.500461
15148
臺北市延平北路七段18-2號中央分隔島
25.095187
121.49791
15149
臺北市延平北路七段18-2號中央分隔島
25.095187
121.49791
15150
臺北市承德路五段與中正路口中央分隔島
25.096101
121.513887
15151
臺北市承德路四段與通河街口往大同方向
25.091019
121.516424
15152
臺北市中山北路五段與通河街口
25.102637
121.526431
15153
臺北市承德路四段150號前分隔島燈桿
25.085181
121

臺北市師大路水源路口東方號誌桿
25.026565
121.528494
15318
臺北市師大路水源路口東方號誌桿
25.026565
121.528494
15319
臺北市師大路、汀州路口
25.026565
121.528494
15320
臺北市師大路、汀州路口
25.026565
121.528494
15321
臺北市師大路、汀州路口
25.026565
121.528494
15322
臺北市師大路、汀州路口
25.026565
121.528494
15323
臺北市師大路、汀州路口
25.026565
121.528494
15324
臺北市羅斯福路四段21號水源市場旁
25.014248
121.535197
15325
臺北市羅斯福路四段21號水源市場旁
25.014248
121.535197
15326
臺北市中正橋上
24.98972977
121.57004896
15327
臺北市中正橋上
24.98972977
121.57004896
15328
臺北市中正橋上
24.98972977
121.57004896
15329
臺北市南海路與泉州街
25.032934
121.518129
15330
臺北市南海路與泉州街
25.032934
121.518129
15331
臺北市南海路與泉州街
25.032934
121.518129
15332
臺北市南海路與泉州街
25.032934
121.518129
15333
臺北市松隆路基隆路口西方門型桿
25.048437
121.575994
15334
臺北市松隆路基隆路口西方門型桿
25.048437
121.575994
15335
臺北市松隆路基隆路口西方門型桿
25.048437
121.575994
15336
臺北市松隆路基隆路口西方門型桿
25.048437
121.575994
15337
臺北市松隆路基隆路口西方門型桿
25.048437
121.575994
15338
臺北市松隆路基隆路口東方門型桿
25.048437
121.575994
15339
臺北市松隆路基隆路口東方門型桿
25.048437
121.575994
15340
臺北市松隆路基隆路口東方門型桿
25.048437
121.575994
15341
臺北市松隆路基隆路口東方門型桿
